In [ ]:
import numpy as np
import os
import keras

In [ ]:
!git clone https://github.com/RishiNandha/AWS_Dataset

Cloning into 'AWS_Dataset'...
remote: Enumerating objects: 5414, done.
remote: Counting objects: 100% (1473/1473), done.
remote: Compressing objects: 100% (1457/1457), done.
remote: Total 5414 (delta 19), reused 1468 (delta 16), pack-reused 3941
Receiving objects: 100% (5414/5414), 93.88 MiB | 23.09 MiB/s, done.
Resolving deltas: 100% (20/20), done.
Updating files: 100% (5340/5340), done.


In [ ]:
from keras.applications.mobilenet_v2 import MobileNetV2

In [ ]:
from imageio import imread
from skimage.transform import resize
from matplotlib.pyplot import imshow
from keras.applications.mobilenet_v2 import preprocess_input

In [ ]:
objects = {"wood":[1,0,0,0,0,0],"glass":[0,1,0,0,0,0],"metal":[0,0,1,0,0,0],"paper":[0,0,0,1,0,0],"plastic":[0,0,0,0,1,0],"mixed":[0,0,0,0,0,1]}

In [ ]:
total_objects=0
for o in objects:
  for imgs in os.scandir('AWS_Dataset/Dataset/{}'.format(o)):
    total_objects += 1

In [ ]:
total_objects

5338

In [ ]:
data = np.empty((3000,224,224,3))
out = np.empty((3000,6))
vdata = np.empty((600,224,224,3))
vout = np.empty((600,6))

In [ ]:
print(data)

[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]


 [[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0

In [ ]:
print(data.shape)

(3000, 224, 224, 3)


In [ ]:

  print("using data set")
  i=0
  n=0
  s=0
  for o in objects:
    n+=1
    print(o)
    print(n)
    for imgs in os.scandir('AWS_Dataset/Dataset/{}'.format(o)):
      im = imread(imgs.path)
      im = preprocess_input(im)
      im = resize(im,output_shape=(224, 224))
      if im.shape == (224,224):
        img_array_expanded = np.expand_dims(im, axis=-1)
        im3 = np.repeat(img_array_expanded, 3, axis=-1)
      else:
        im3 = im

      if(s<n*100):
        vdata[s] = im3[:,:,:3]
        vout[s] = objects[o]
        s+=1
      elif(i<n*500):

        data[i] = im3[:,:,:3]
        out[i] = objects[o]

        i+=1


using data set
wood
1


<ipython-input-11-23f454ebe5dc>:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  im = imread(imgs.path)


glass
2
metal
3
paper
4
plastic
5
mixed
6


In [ ]:
im.shape

(224, 224, 3)

In [ ]:
model = MobileNetV2(weights = 'imagenet')

14536120/14536120 [==============================] - 0s 0us/step


In [ ]:

from tensorflow.keras import layers, Model
from keras.layers import Dense
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam

In [ ]:
aws = Dense(6, activation='softmax')

In [ ]:
aws = aws(model.layers[-2].output)

In [ ]:
from keras import Model
aws_input = model.input
aws_model = Model(inputs = aws_input, outputs=aws)
for layer in aws_model.layers[:-4]:
    layer.trainable = False

In [ ]:
optimizer = Adam(learning_rate=0.009)

In [ ]:
aws_model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint('ckpt.keras',monitor = 'val_accuracy', verbose = 1, save_best_only=True, mode ='max')

stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
try:
 aws_model.load_weights('ckpt.keras')
except:print('creating new')

creating new


In [ ]:


aws_model.fit(x = data, y = out,verbose=2,validation_data=(vdata,vout),batch_size=40,epochs=10,callbacks = [checkpoint,stop])

Epoch 1/10

Epoch 1: val_accuracy improved from -inf to 0.73833, saving model to ckpt.keras
75/75 - 15s - loss: 0.8868 - accuracy: 0.7023 - val_loss: 0.8793 - val_accuracy: 0.7383 - 15s/epoch - 194ms/step
Epoch 2/10

Epoch 2: val_accuracy improved from 0.73833 to 0.78667, saving model to ckpt.keras
75/75 - 6s - loss: 0.4344 - accuracy: 0.8510 - val_loss: 0.6757 - val_accuracy: 0.7867 - 6s/epoch - 73ms/step
Epoch 3/10

Epoch 3: val_accuracy improved from 0.78667 to 0.81333, saving model to ckpt.keras
75/75 - 5s - loss: 0.3441 - accuracy: 0.8870 - val_loss: 0.5451 - val_accuracy: 0.8133 - 5s/epoch - 68ms/step
Epoch 4/10

Epoch 4: val_accuracy improved from 0.81333 to 0.82667, saving model to ckpt.keras
75/75 - 5s - loss: 0.2526 - accuracy: 0.9147 - val_loss: 0.5768 - val_accuracy: 0.8267 - 5s/epoch - 66ms/step
Epoch 5/10

Epoch 5: val_accuracy did not improve from 0.82667
75/75 - 5s - loss: 0.2142 - accuracy: 0.9203 - val_loss: 0.6252 - val_accuracy: 0.8217 - 5s/epoch - 62ms/step
Epoch 6

In [ ]:
del data
del out
import gc
gc.collect()
predictions=aws_model.predict(vdata)

19/19 [==============================] - 3s 83ms/step


In [ ]:
Ob=[[1,0,0,0,0,0] ,[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]]

In [ ]:
for t in range(0,600):
  predictions[t]=Ob[np.argmax(predictions[t])]
from sklearn.metrics import f1_score
print(f1_score(vout,predictions,average=None))

[0.94416244 0.84057971 0.80851064 0.81512605 0.80808081 0.70930233]


In [ ]:
#keras.backend.set_value(aws_model.optimizer.learning_rate, 0.005)
#aws_model.fit(x = data, y = out ,verbose=2,validation_data=(vdata,vout),initial_epoch=5,epochs=15,callbacks = [checkpoint,stop])

In [ ]:

mismatch = np.zeros(6,)
l=0
pos = 0;
for prediction in predictions:
  if np.argmax(prediction) != np.argmax(vout[l]):
    mismatch[np.argmax(vout[l])] += 1
    print(predictions[l]);
  l+=1

[0. 0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0. 0.]
[0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 1. 0.]
[0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0. 0.]
[0. 0. 1. 0. 0. 0.]
[0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 1. 0.]
[0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0.]
[1. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 0. 1.]
[0. 1. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0.]
[0. 1. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0.]
[0. 1. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0.]
[0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1.]
[0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 1.]
[0. 0. 0. 1. 0. 0.]
[0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1.]
[0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 0. 1.]
[0. 0. 0. 1. 0. 0.]
[0. 1. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 1.]
[0. 1. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 0. 1.]
[0. 1. 0. 0. 0. 0.]


In [ ]:
aws_model.save('aws_model')

In [ ]:
aws_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        